## 【第7回】簡単なゲーム作成

プログラミング演習TAの皆様<br>
担当の石川です。<br>
資料公開しました。と言っても今回は今まで出てきたのを組み合わせじゃんけんゲームをつくるので新規要素はないです。<br>
独自機能は例を挙げるとそればかりで面白くないので具体的な例は挙げませんでしたが、聞かれたら3連勝や難易度調整など具体的な例を教えてください。<br>

#### 【A課題】以下の仕様の1回だけじゃんけんを行うプログラムを作成せよ
* コンピュータは乱数でグー・チョキ・パーのいずれかを出す
* プレイヤーは数字でじゃんけんの手を入力する
* 勝・負・あいこを表示

##### じゃんけん勝敗分け
* 人間・PCの手：それぞれ3通り 3×3通りの場合分けが必要
* 愚直に3×3=9通りの場合分けする； **推奨しない** <br>
二重match文or ifで全9通りを愚直に場合分け
* PCと人間の手が同じ場合：あいこ<br>

|人間         |PC         |勝ち|
|:-----------|:----------|----|
|`0` : グー  |`2` : パー  |PC |
|`1` : チョキ|`0` : グー  |^  |
|`2` : パー  |`1` : チョキ|^  |
|`0` : グー  |`1` : チョキ|人間|
|`1` : チョキ|`2` : パー  |^  |
|`2` : パー  |`0` : グー  |^  |

In [1]:
import random

print("じゃんけんをしましょう！")
print("0: グー, 1: チョキ, 2: パー")

player_hand = int(input("あなたの手を数字で入力してください: "))
print(f"あなたの手を数字で入力してください: {player_hand}") ### Debug
pc_hand = random.randint(0, 2)

# 出した手を表示
if player_hand == 0:
    print("あなたの手: グー")
elif player_hand == 1:
    print("あなたの手: チョキ")
elif player_hand == 2:
    print("あなたの手: パー")
else:
    print("無効な入力です。0～2の数値を入力してください。")

if pc_hand == 0:
    print("コンピューターの手: グー")
elif pc_hand == 1:
    print("コンピューターの手: チョキ")
else:
    print("コンピューターの手: パー")

# 勝敗判定
if player_hand == pc_hand:
    print("あいこです！")
elif (player_hand + 1) % 3 == pc_hand:
    print("あなたの勝ちです！")
else:
    print("あなたの負けです")

じゃんけんをしましょう！
0: グー, 1: チョキ, 2: パー
あなたの手を数字で入力してください: 2
あなたの手: パー
コンピューターの手: グー
あなたの勝ちです！


#### 【B課題】 以下の動作を満たすじゃんけんゲームを作成せよ
* 最初にコメントで実装した独自機能にコメントをつけること
* 勝負回数 `n` を入力
* コンピュータは乱数でグー・チョキ・パーのいずれかを出す
* プレイヤーは数字でじゃんけんの手を入力する
* あいこの場合は勝負回数n回に含めず、勝敗が決まるまで繰り返す
* 最終的な勝敗 [ `i` 勝, `j` 引分, `n-i` 敗] を表示する

In [2]:
"""
独自機能
PCの手の3パターン（ランダム、前回と同じ、必勝）を実装
誤入力時に再入力を促す
プレイヤーが3連勝したら即勝利
難易度調整機能を追加
"""

import random  # 標準の乱数モジュール

print("ジャンケンゲームを開催します")

# ★ 難易度選択
print("難易度を選んでください（1:やさしい, 2:ふつう, 3:むずかしい）")
while True:
    try:
        difficulty = int(input("難易度: "))
        print(f"難易度: {difficulty}") ### Debug
        if 1 <= difficulty <= 3:
            break
        else:
            print("1〜3の整数を入力してください。")
    except ValueError:
        print("整数を入力してください。")

# ★ 勝負回数の入力
while True:
    try:
        total_rounds = int(input("何回勝負にしますか？ "))
        print(f"何回勝負にしますか？ {total_rounds}") ### Debug
        if total_rounds > 0:
            break
        else:
            print("正の整数を入力してください。")
    except ValueError:
        print("整数を入力してください。")

print("数字とじゃんけんの対応は次の通りです。")
print("0:グー, 1:チョキ, 2:パー")

player_win_count = 0
draw_count = 0
match_count = 0
previous_pc_hand = -1  # 前回のPCの手を記録
consecutive_wins = 0    # 連勝回数を記録

while match_count < total_rounds:
    # プレイヤーの手を入力
    while True:
        try:
            player_hand = int(input("あなたの手を入力してください: "))
            print(f"あなたの手を入力してください: {player_hand}") ### Debug
            if 0 <= player_hand <= 2:
                break
            else:
                print("入力ミスです！0〜2の整数を入力してください。")
        except ValueError:
            print("整数を入力してください。")

    # あなたの手を表示
    if player_hand == 0:
        print("あなたの手はグー")
    elif player_hand == 1:
        print("あなたの手はチョキ")
    else:
        print("あなたの手はパー")

    # ★ PCの戦略パターン決定
    # 0:ランダム, 1:前回と同じ, 2:プレイヤーに勝つ手
    if difficulty == 1:
        weights = [0.7, 0.2, 0.1]  # 初級：ほぼランダム
    elif difficulty == 2:
        weights = [0.4, 0.3, 0.3]  # 中級：バランス
    else:
        weights = [0.2, 0.2, 0.6]  # 上級：勝つ手が多い

    r = random.random()
    if r < weights[0]:
        pc_pattern = 0
    elif r < weights[0] + weights[1]:
        pc_pattern = 1
    else:
        pc_pattern = 2

    # ★ PCの手を決定
    if pc_pattern == 0 or previous_pc_hand == -1:
        pc_hand = random.randint(0, 2)
    elif pc_pattern == 1:
        pc_hand = previous_pc_hand
    else:
        pc_hand = (player_hand + 1) % 3  # プレイヤーに勝つ手

    previous_pc_hand = pc_hand  # 前回のPCの手を更新

    # PCの手を表示
    if pc_hand == 0:
        print("PCの手はグー")
    elif pc_hand == 1:
        print("PCの手はチョキ")
    else:
        print("PCの手はパー")

    # 勝敗判定
    if pc_hand == player_hand:
        print("引き分けです。")
        draw_count += 1
        consecutive_wins = 0
    elif (player_hand + 1) % 3 == pc_hand:
        print("あなたの勝ち！")
        player_win_count += 1
        match_count += 1
        consecutive_wins += 1
    else:
        print("PCの勝ち...")
        match_count += 1
        consecutive_wins = 0

    # ★ 3連勝で即勝利
    if consecutive_wins == 3:
        print("\n★★★ 3連勝達成！あなたの完全勝利です！★★★")
        break

# ★ 結果表示
player_lose_count = match_count - player_win_count
print(f"\n結果: {player_win_count}勝, {draw_count}引き分け, {player_lose_count}敗")

if consecutive_wins == 3:
    print("あなたが3連勝で勝者です！")
elif player_win_count > player_lose_count:
    print("あなたが勝者です！")
elif player_win_count == player_lose_count:
    print("引き分けです。")
else:
    print("PCが勝者です。")

ジャンケンゲームを開催します
難易度を選んでください（1:やさしい, 2:ふつう, 3:むずかしい）
難易度: 3
何回勝負にしますか？ 5
数字とじゃんけんの対応は次の通りです。
0:グー, 1:チョキ, 2:パー
あなたの手を入力してください: 0
あなたの手はグー
PCの手はグー
引き分けです。
あなたの手を入力してください: 1
あなたの手はチョキ
PCの手はパー
あなたの勝ち！
あなたの手を入力してください: 2
あなたの手はパー
PCの手はグー
あなたの勝ち！
あなたの手を入力してください: 0
あなたの手はグー
PCの手はグー
引き分けです。
あなたの手を入力してください: 1
あなたの手はチョキ
PCの手はグー
PCの勝ち...
あなたの手を入力してください: 2
あなたの手はパー
PCの手はグー
あなたの勝ち！
あなたの手を入力してください: 0
あなたの手はグー
PCの手はチョキ
あなたの勝ち！

結果: 4勝, 2引き分け, 1敗
あなたが勝者です！
